In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

     |████████████████████████████████| 968 kB 13.4 MB/s 
     |████████████████████████████████| 359 kB 57.8 MB/s 
     |████████████████████████████████| 138 kB 52.8 MB/s 
     |████████████████████████████████| 55 kB 3.8 MB/s 
     |████████████████████████████████| 3.6 MB 46.1 MB/s 
     |████████████████████████████████| 58 kB 5.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.

In [ ]:
#Task1
#Crawling 100 comments
import time
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
import re
from collections import Counter
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Flatten
from tensorflow.keras import layers , activations , models , preprocessing , utils

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [ ]:
import pandas as pd
df1 = pd.read_csv('/content/data_comment_1.csv',names = ["Comment","Label"])
df1 = pd.DataFrame(df1)
df1["Label"] = df1["Label"].replace({'Disgust':0,'Enjoyment':1,'Anger':2,'Surprise':3,'Sadness':4,'Fear':5,'Other':6})
df1.head()

,Comment,Label
0,Âm thanh hay quá ❤️ Cảm ơn chương trình Xuân H...,1
1,"Cách đây 27 năm trước, mẹ mình yêu chú, thời ấ...",4
2,Mình và chồng sắp cưới 4 năm trước có ước địn...,1
3,Mình nghe bài này thì nhận được tin nhắn của n...,4
4,Lyric: Dưới cơn mưa phùn nhẹ. Có hai kẻ khờ ch...,6


In [ ]:
df2 = pd.read_csv('/content/data_comment_2.csv',names = ["Comment","Label"])
df2['Label'] = df2['Label'].replace({'Disgust':0,'Enjoyment':1,'Anger':2,'Surprise':3,'Sadness':4,'Fear':5,'Other':6})
df2.head()

,Comment,Label
0,Âm nhạc là một món quà. Hôm nay mình nghĩ rất ...,1
1,MV Sài Gòn Đau Lòng Quá của Hứa Kim Tuyền ft H...,1
2,tik tok lại đưa e tới,6
3,Có ai khóc khi nghe lại đoạn điệp khúc lần 2 k...,4
4,Mong ai đã và đang trải qua cảm giác này sớm t...,4


In [ ]:
df3 = pd.read_csv('/content/data_comment_3.csv',names = ["Comment","Label"])
df3['Label'] = df3['Label'].replace({'Disgust':0,'Enjoyment':1,'Anger':2,'Surprise':3,'Sadness':4,'Fear':5,'Other':6})
df3.head()

,Comment,Label
0,Có ai nghe đến đoạn nhạc cuối trong MV hông 😘 ...,1
1,"Dễ nghe ấy chứ, nghe nhiều chắc chắn sẽ nghiện...",1
2,"Chúc mừng Phí Phương Anh, ca khúc quá tuyệt vờ...",1
3,mình thấy đoạn bridge của Rin9 đoạn em là nỗi ...,3
4,Giá như có thể gặp anh ở một thời điểm khác......,4


In [ ]:
def clean_data(text):
    text_ = text.lower()
    text_ = re.sub(r'([^\s\w]|_)+', '', text_)
    return text_.strip()

In [ ]:
#Task2
#Read data
#Clean data train
df_train = pd.read_excel('/content/train_nor_811.xlsx',engine = 'openpyxl')
df_train = pd.DataFrame(df_train)
df_train = df_train[['Sentence','Emotion']] 
df_train= df_train.rename(columns = {'Sentence': 'Comment', 'Emotion': 'Label'}, inplace = False)
df_train['Label'] = df_train['Label'].map({'Disgust':0,'Enjoyment':1,'Anger':2,'Surprise':3,'Sadness':4,'Fear':5,'Other':6})
for i in df_train['Comment']:
    clean_data(i)
df_train

,Comment,Label
0,cho mình xin bài nhạc tên là gì với ạ,6
1,cho đáng đời con quỷ . về nhà lôi con nhà mày ...,0
2,lo học đi . yêu đương lol gì hay lại thích học...,0
3,uớc gì sau này về già vẫn có thể như cụ này :)),1
4,mỗi lần có video của con là cứ coi đi coi lại ...,1
...,...,...
5543,đường của nhà cụ hay sao mà cụ cấm người ta đỗ...,0
5544,nhìn mặt héo queo luôn,6
5545,tao đi xe máy mỗi lần muốn để xe đi đâu là phi...,6
5546,thích thân hình boss rồi nhan 😌,1


In [ ]:
counter = Counter()
y_train = df_train['Label'].values
X_train = df_train.iloc[:,0].values
counter.update(y_train)
print(counter)
print(len(X_train))

Counter({1: 1558, 0: 1071, 6: 1021, 4: 947, 2: 391, 5: 318, 3: 242})
5548


In [ ]:
#Read data test
df_test = pd.read_excel('/content/valid_nor_811.xlsx',engine='openpyxl')
df_test = pd.DataFrame(df_test)
df_test = df_test[['Sentence','Emotion']] 
df_test= df_test.rename(columns = {'Sentence': 'Comment', 'Emotion': 'Label'}, inplace = False)
df_test['Label'] = df_test['Label'].map({'Disgust':0,'Enjoyment':1,'Anger':2,'Surprise':3,'Sadness':4,'Fear':5,'Other':6})
for i in df_test['Comment']:
    clean_data(i)
df_test

,Comment,Label
0,"tính tao tao biết , chẳng có chuyện gì có thể ...",6
1,"lại là lào cai , tự hào quê mình quá :))",1
2,bị từ chối rồi,4
3,tam đảo trời đẹp các mem à,1
4,đọc bình luận của thằng đó không thiếu chữ nào 😂😂,6
...,...,...
681,nhọc lòng :(((,4
682,tao đã rất muốn làm bạn tốt anh em tri kỉ với ...,4
683,cơm tấm ở ngoài cũng như này . 😡,0
684,chất liệu đỉnh quá,3


In [ ]:
counter_test = Counter()
y_test = df_test['Label'].values
X_test = df_test.iloc[:,0].values
counter_test.update(y_test)
print(counter_test)
print(len(X_test))

Counter({1: 214, 6: 141, 0: 135, 4: 86, 2: 49, 5: 31, 3: 30})
686


In [ ]:
#Chuyển text về ma trận để phù hợp với model
# sentences_train = [word_tokenize(i) for i in X_train]
# tokens = Tokenizer()
# tokens.fit_on_texts(sentences_train)
# X_train_matrix = tokens.texts_to_matrix(sentences_train,mode = 'binary')
# print(X_train_matrix.shape)

# sentences_test = [word_tokenize(i) for i in X_test]
# tokens_test = Tokenizer()
# tokens_test.fit_on_texts(sentences_test)
# X_test_matrix = tokens_test.texts_to_matrix(sentences_test,mode = 'binary')
# X_test_matrix = pad_sequences(X_test_matrix,6383)
# print(X_test_matrix.shape)

In [ ]:
data = []
for i in df_train['Comment']:
    data.append(i)
for i in df_test['Comment']:
    data.append(i)
    
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True,split=' ')
tokenizer.fit_on_texts(data)

X = tokenizer.texts_to_sequences(data)
X = pad_sequences(X)



In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D,Flatten,Embedding

vocab_size=len(tokenizer.word_index)+1
embedding_dim = 128
hidden_size = 200
batch_size = 32

model = Sequential()
model.add(layers.Embedding(vocab_size,embedding_dim,input_length = X.shape[1]))
model.add(layers.LSTM(hidden_size))
model.add(layers.Dense(128,activation = 'sigmoid'))
model.add(layers.Dense(7,activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 154, 128)          594816    
                                                                 
 lstm (LSTM)                 (None, 200)               263200    
                                                                 
 dense (Dense)               (None, 128)               25728     
                                                                 
 dense_1 (Dense)             (None, 7)                 903       
                                                                 
Total params: 884,647
Trainable params: 884,647
Non-trainable params: 0
_________________________________________________________________


In [ ]:
label = [i for i in df_train['Label']]
for i in df_test['Label']:
    label.append(i)

In [ ]:
from sklearn.preprocessing import LabelEncoder

y = tf.keras.utils.to_categorical(label)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
model.fit(X_train, y_train,epochs=50,verbose=1,validation_data=(X_test, y_test),batch_size=20)

Epoch 1/50
250/250 [==============================] - 94s 366ms/step - loss: 1.6974 - accuracy: 0.3277 - val_loss: 1.4851 - val_accuracy: 0.4427
Epoch 2/50
250/250 [==============================] - 90s 360ms/step - loss: 1.2277 - accuracy: 0.5498 - val_loss: 1.3149 - val_accuracy: 0.5148
Epoch 3/50
250/250 [==============================] - 90s 361ms/step - loss: 0.8839 - accuracy: 0.6858 - val_loss: 1.3287 - val_accuracy: 0.5269
Epoch 4/50
250/250 [==============================] - 90s 360ms/step - loss: 0.6292 - accuracy: 0.7895 - val_loss: 1.4692 - val_accuracy: 0.5156
Epoch 5/50
250/250 [==============================] - 89s 356ms/step - loss: 0.4528 - accuracy: 0.8556 - val_loss: 1.6177 - val_accuracy: 0.5277
Epoch 6/50
250/250 [==============================] - 90s 358ms/step - loss: 0.3255 - accuracy: 0.8995 - val_loss: 1.7624 - val_accuracy: 0.5180
Epoch 7/50
250/250 [==============================] - 88s 351ms/step - loss: 0.2434 - accuracy: 0.9252 - val_loss: 1.9651 - val_ac

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
#dùng data crawl ở task1
label_test = [i for i in df1['Label']]
for i in df2['Label']:
    label_test.append(i)
for i in df3['Label']:
    label_test.append(i)
data_test = [i for i in df1['Comment']]
for i in df2['Comment']:
    data_test.append(i)
for i in df3['Comment']:
    data_test.append(i)
    
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True,split=' ')
tokenizer.fit_on_texts(data_test)

X_test = tokenizer.texts_to_sequences(data_test)
X_test = pad_sequences(X_test)
y_test = tf.keras.utils.to_categorical(label_test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
model.save('LSTM')